In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

In [4]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-02-13 17:35:26,915 INFO: Initializing external client
2025-02-13 17:35:26,918 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-13 17:35:28,713 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1211551
Fetching data from 2025-01-15 22:35:26.912840+00:00 to 2025-02-13 21:35:26.912840+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.41s) 


In [5]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-01-15 23:00:00+00:00,2,0
1,2025-01-16 00:00:00+00:00,2,0
2,2025-01-16 01:00:00+00:00,2,0
3,2025-01-16 02:00:00+00:00,2,0
4,2025-01-16 03:00:00+00:00,2,0
...,...,...,...
178508,2025-02-13 17:00:00+00:00,263,162
178509,2025-02-13 18:00:00+00:00,263,146
178510,2025-02-13 19:00:00+00:00,263,136
178511,2025-02-13 20:00:00+00:00,263,123


In [6]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 178513 entries, 0 to 184934
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         178513 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  178513 non-null  int32                  
 2   rides               178513 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.1 MB


In [7]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [8]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 178513 entries, 0 to 184934
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         178513 non-null  datetime64[us]
 1   pickup_location_id  178513 non-null  int32         
 2   rides               178513 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.1 MB


In [9]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

Skipping location_id 109: Not enough data to create even one window.


In [10]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,2,67,2025-02-06 05:00:00
1,48,25,8,2,15,10,16,12,1,21,...,5,21,37,3,17,69,72,18,148,2025-01-16 18:00:00
2,14,1,33,44,36,1,0,77,40,13,...,0,0,42,16,39,87,38,1,13,2025-01-30 09:00:00
3,0,1,2,0,2,1,0,0,1,0,...,0,0,1,0,0,0,2,0,218,2025-01-28 05:00:00
4,0,0,1,0,1,0,1,1,1,0,...,0,0,0,0,0,1,0,0,202,2025-01-26 21:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,102,2025-02-06 19:00:00
252,14,1,18,113,2,389,2,26,18,117,...,14,39,94,13,96,30,127,58,114,2025-02-03 18:00:00
253,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,99,2025-02-13 13:00:00
254,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-02-03 13:00:00


In [11]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-02-13 17:37:17,628 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-13 17:37:17,638 INFO: Initializing external client
2025-02-13 17:37:17,639 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-13 17:37:19,110 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1211551
Fetching data from 2025-01-15 22:37:17.628670+00:00 to 2025-02-13 21:37:17.628670+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (9.11s) 
Skipping location_id 109: Not enough data to create even one window.


In [12]:
current_date

Timestamp('2025-02-13 22:37:17.628670+0000', tz='Etc/UTC')

In [13]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,2,2025-02-12 23:00:00
1,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,3,2025-02-12 23:00:00
2,0,0,0,0,0,0,0,5,5,8,...,4,5,6,10,5,5,9,5,4,2025-02-12 23:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,2025-02-12 23:00:00
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-02-12 23:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,0,0,0,0,0,0,0,3,0,0,...,0,0,0,0,1,0,1,0,259,2025-02-12 23:00:00
252,0,0,0,0,1,0,0,0,2,3,...,3,1,1,1,0,4,0,1,260,2025-02-12 23:00:00
253,0,0,0,0,0,0,4,2,6,12,...,36,27,50,60,41,27,30,17,261,2025-02-12 23:00:00
254,0,0,0,0,2,4,14,75,138,162,...,126,90,127,162,96,56,47,22,262,2025-02-12 23:00:00


In [14]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-02-13 17:41:32,318 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-13 17:41:32,334 INFO: Initializing external client
2025-02-13 17:41:32,337 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-13 17:41:33,715 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1211551


In [15]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [16]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,2.0
3,5,0.0
4,6,0.0
...,...,...
251,259,0.0
252,260,1.0
253,261,10.0
254,262,10.0


In [17]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([132, 249, 230, 161,  79, 142,  48, 114, 138, 163])